# Prompt Optimisation

As your system built on AgentX goes to production, whether it is using the Agent class or publishing Tool to the store, you will accumulate more and more feedback data. Utilising these data, you can tune the prompt and the inference hyperparameters to achieve elevated performance.

This notebook demonstrate how to tune prompt and the inference hyperparameters to better predict financial news sentiment.

In [20]:
# Get a dataset with QQQ news and price data.
# sourced from https://www.kaggle.com/datasets/miguelaenlle/massive-stock-news-analysis-db-for-nlpbacktests
from pandas import read_csv
from rich import print as rich_print

dataset = read_csv(
    'data/qqq_sentiment.csv'
)

stats = dataset.describe()
stats = stats.to_dict()
rich_print(stats)

{
    'price_after_a_week': {
        'count': 830.0,
        'mean': 0.335120215900884,
        'std': 2.6381432754727108,
        'min': -11.63959526159922,
        '25%': -0.7588281764284638,
        '50%': 0.5679872906183354,
        '75%': 1.7928792178315212,
        'max': 12.65985281814521
    }
}

In [21]:
# We will define negative as the bottom 25% of one week percentage change
# and top 25% as positive.

stats = dataset.describe()

def classify_movement(price):
    if price <= stats['price_after_a_week']['25%']:
        return 'negative'
    elif price >= stats['price_after_a_week']['75%']:
        return 'positive'
    else:
        return 'neutral'

dataset['price_movement'] = dataset.price_after_a_week.apply(classify_movement)

In [22]:
# Use an easily defined sentiment classification agent to get a baseline

from agentx.agent import Agent
from agentx.schema import GenerationConfig, Message, Content
from dotenv import load_dotenv
from random import sample
from typing import List, Union, Literal
from pydantic import BaseModel
import os
import asyncio

load_dotenv()

class PricePrediction(BaseModel):
    movement:Literal['positive', 'negative', 'neutral']

generation_config = GenerationConfig(
    api_type='azure',
    api_key=os.environ.get('AZURE_OPENAI_KEY'),
    base_url=os.environ.get('AZURE_OPENAI_ENDPOINT'),
    azure_deployment='gpt-35',
)

sentiment_classification_agent = Agent(
    name='financial_news_sentiment_classification',
    generation_config=generation_config,
    system_prompt='''You will predict the price movement of QQQ based on news headlines.
The price movement can be positive, negative, or neutral.
If the price movement is positive, the price of QQQ should go up substantially in a week.
Vice versa, if the price movement is negative, the price of QQQ should go down substantially in a week.''',
)

# sample a small test set
test_set = sample(dataset.to_dict(orient='records'), 50)
rich_print(test_set)

[
    {
        'title': 'Benzinga Pro Mid-Day Price Check For Major Averages: Dow 30 Down 34 (1.2%), Russell 2000 Down 20 
(1.3%), S&P 500 Down 390(1.5%), Nasdaq Down 147 (2.0%)',
        'date': '2018-11-12',
        'stock': 'QQQ',
        'price_after_a_week': -2.567188089133793,
        'price_movement': 'negative'
    },
    {
        'title': "Clear Sailing Ahead: Fed's Yellen Says No Short-Term Obstacles For U.S. Economy",
        'date': '2017-01-13',
        'stock': 'QQQ',
        'price_after_a_week': 0.0730724237391217,
        'price_movement': 'neutral'
    },
    {
        'title': 'Benzinga Pro Morning Price Check For Major Averages: Nasdaq Up 27 (0.4%), S&P 500 Up 4 (0.1%), 
Dow 30 Up 17 (0.1%), Russell 2000 Flat',
        'date': '2018-06-26',
        'stock': 'QQQ',
        'price_after_a_week': -0.7380740096093563,
        'price_movement': 'neutral'
    },
    {
        'title': 'Benzinga Pro Mid-Day Price Check For Major Averages: Nasdaq Down 0.7%, S&P 500 Down 0.8%, Russell
Down 0.9%, Dow Jones Down 1.1%',
        'date': '2019-08-12',
        'stock': 'QQQ',
        'price_after_a_week': 2.213174324496625,
        'price_movement': 'positive'
    },
    {
        'title': 'Benzinga Pro Mid-Day Price Check For Major Averages: Dow Jones Down 0.8%, Russell Down 0.5%, S&P 
500 Down 6.5%, Nasdaq Down 0.5%',
        'date': '2019-04-09',
        'stock': 'QQQ',
        'price_after_a_week': 1.09497183640442,
        'price_movement': 'neutral'
    },
    {
        'title': 'How Latin America Lived Election Day: Between Gossip And Outright Dislike',
        'date': '2016-11-09',
        'stock': 'QQQ',
        'price_after_a_week': -0.6289825647431777,
        'price_movement': 'neutral'
    },
    {
        'title': 'Benzinga Pro Mid-Day Price Check For Major Averages: Nasdaq Up 36 (0.5%), Russell 2000 Down 3 
(0.2%), S&P 500 Down 8 (0.3%),  Dow 30 Down 136 (0.5%)',
        'date': '2018-10-22',
        'stock': 'QQQ',
        'price_after_a_week': -6.141111928213163,
        'price_movement': 'negative'
    },
    {
        'title': 'Less Hawkish Tone: Rates Rise Again, But 2019 Might See Fewer Hikes',
        'date': '2018-12-19',
        'stock': 'QQQ',
        'price_after_a_week': -1.3395405509580138,
        'price_movement': 'negative'
    },
    {
        'title': 'Benzinga Pro Mid-Day Price Check For Major Averages: Nasdaq Up 20 (0.3%),S&P 500 Flat, Dow 30 
Down 18 (0.1%), Russell 2000 Down 6 (0.6%)',
        'date': '2018-06-13',
        'stock': 'QQQ',
        'price_after_a_week': 0.8133278029047036,
        'price_movement': 'neutral'
    },
    {
        'title': 'Benzinga Pro Mid-Day Price Check For Major Averages: Russell 2000 Up 18 (1.3%), S&P 500 Up 34 
(1.3%), Dow 30 Up 312 (1.3%), Nasdaq Up 86 (1.2%)',
        'date': '2019-01-18',
        'stock': 'QQQ',
        'price_after_a_week': -0.0605180030257232,
        'price_movement': 'neutral'
    },
    {
        'title': 'Benzinga Pro Mid-Day Price Check For Major Averages: Russell 2000 Up 13 (0.8%), Nasdaq Up 50 
(0.7%), S&P 500 Up 12 (0.4%), Dow 30 Up 54 (0.2%)',
        'date': '2018-05-16',
        'stock': 'QQQ',
        'price_after_a_week': 0.3669132528562775,
        'price_movement': 'neutral'
    },
    {
        'title': 'Benzinga Pro Afternoon Price Check For Major Averages: Russell 2000 Down 25 (1.7%), Dow 30 Down 
499 (2.0%), S&P 500 Down 52 (2.0%), Nasdaq Down 260 (2.3%)',
        'date': '2018-12-14',
        'stock': 'QQQ',
        'price_after_a_week': -8.387133618237726,
        'price_movement': 'negative'
    },
    {
        'title': "US Indices are lower for the session amid investor concerns following comments from Fed Chair 
Powell suggesting high uncertainty and 'significant' downside risk.",
        'date': '2020-05-13',
        'stock': 'QQQ',
        'price_after_a_week': 5.493755457167043,
        'price_movement': 'positive'
    },
    {
        'title': 'Pro: FAANG Stocks Could Have 30-40% Downside Potential

In [23]:

responses = await asyncio.gather(*[
    sentiment_classification_agent.a_generate_response(
        messages=[
            Message(
                role='user',
                content=Content(
                    text=datum['title'],
                )
            )
        ],
        output_model=PricePrediction
    ) for datum in test_set
])

responses = [response for response in responses if response != []]

In [26]:
# Let's check how well the agent did
import numpy as np

def metric(predicted:List[PricePrediction], actual:List[PricePrediction]):
    if len(predicted) != len(actual):
        raise ValueError('predicted and actual must be the same length')
    
    value_map = {
        'positive': 1,
        'negative': -1,
        'neutral': 0
    }

    p = np.array([value_map[p.movement] for p in predicted])
    a = np.array([value_map[a.movement] for a in actual])
    
    error = np.square(p - a).mean()
    return error

metric(
    [PricePrediction.model_validate_json(response.content.text) for response in responses], 
    [PricePrediction(movement=datum.get('price_movement')) for datum in test_set]
)

1.14

An mean square error of 1.14 basically mean the model is doing nothing at predicting the sentiment. The actual movement is ever so slightly going opposite of what the model predicted.

In [13]:
from agentx.optimisers import TextualGradientPromptTrainer

reviewer = Agent(
    name='reviewer',
    generation_config=generation_config,
    system_prompt='You are a prompt engineer. Review the given prompt, error samples and give reasons why the prompt have gotten these examples wrong.',
)

def textual_gradient(
    prompt:str,
    input:List[str],
    predicted:List[PricePrediction],
    truth:List[PricePrediction],
) -> List[Message]:
    errors = [
        {
            'input': input,
            'predicted': predicted.movement,
            'truth': truth.movement,
        } for input, predicted, truth in zip(input, predicted, truth) if predicted != truth
    ]

    messages = [
        Message(
            role='user',
            content=Content(
                text='''Current prompt: {prompt}

Errors: {errors}'''.format(prompt=prompt, errors=errors, )
            ),
        )
    ]

    response = reviewer.generate_response(
        messages=messages
    )
    
    return messages + [response]

In [15]:
# test the textual_gradient
gradient = textual_gradient(
    prompt=sentiment_classification_agent.system_prompt,
    input=[datum['title'] for datum in test_set],
    predicted=[Sentiment.model_validate_json(response.content.text) for response in responses],
    truth=[Sentiment(**datum) for datum in test_set],
)

In [18]:
rich_print(gradient[-1].content.text)

The given prompt is asking for the sentiment classification of news article headlines about the QQQ, with the 
sentiment being positive, negative, or neutral. The prompt also states that if the sentiment is positive, the price
of QQQ should go up substantially in a week, and if the sentiment is negative, the price should go down 
substantially in a week.

The errors in the given input-output pairs indicate several issues with the prompt:

1. Limited Context: The prompt does not provide enough context for the model to accurately predict the sentiment. 
Predicting sentiment based solely on headlines without the full content of the news article may not capture the 
complete sentiment.

2. Misalignment of Sentiment and Price Movement: Associating specific sentiments with a guarantee of price movement
(e.g., positive sentiment leading to a substantial price increase) creates an unrealistic expectation. Market 
movements are influenced by a multitude of factors, and predicting price changes solely based on sentiment is 
overly simplistic.

3. Inconsistent Truth Values: The truth values in the error samples do not consistently align with the predicted 
sentiments or the expected price movements. This inconsistency suggests that the ground truth may not reflect a 
direct link between sentiment and subsequent price changes.

4. Overgeneralization: The prompt assumes a deterministic cause-and-effect relationship between sentiment and price
movement, which is overly simplistic for the complexities of financial markets. Stock prices are influenced by a 
wide range of fundamental and technical factors, making it challenging to attribute changes solely to sentiment.

To improve the prompt, it should focus on sentiment analysis of the news article headlines without making explicit 
claims about subsequent price movements. Additionally, providing a larger context or dataset that includes more 
diverse news articles and their corresponding stock price movements would allow for a more realistic sentiment 
analysis task.

In [ ]:
from ray import tune
from ray.tune.schedulers import ASHAScheduler